In [61]:
import json

## LKH Heuristic

In [62]:

import os
from glob import glob
fs = glob("LKH_solutions/*")
tasks = []
lkh_times = []
lkh_objs = []
lkh_opt_gaps = []
for f in fs:
    obj = json.load(open(f))
    time = obj['warmstart_latency']
    objective = obj['current_obj']
    task = os.path.basename(f).split("_solution")[0]
    tasks.append(task)
    lkh_times.append(time)
    lkh_objs.append(int(objective))
    opt = int(open('./instances/opt/{}.opt.tour.txt'.format(task)).readlines()[-1].strip())
    lkh_opt_gaps.append((objective - opt) / opt)

In [63]:
tasks

['d1291', 'dsj1000', 'd15112', 'd2103', 'fl1577']

In [67]:
import pandas as pd
dataframe = pd.DataFrame([lkh_times, lkh_objs, lkh_opt_gaps], columns=tasks, index=['time', 'objective', 'optimality_gaps'])
dataframe

,d1291,dsj1000,d15112,d2103,fl1577
time,1.602132,1.077386e+00,5.242790e+02,4.400274,3.510462
objective,51112.000000,1.869165e+07,1.592268e+06,80883.000000,22305.000000
optimality_gaps,0.006122,1.685835e-03,1.219515e-02,0.005382,0.002517


In [94]:
dataframe.to_latex()

'\\begin{tabular}{lrrrrr}\n\\toprule\n & d1291 & dsj1000 & d15112 & d2103 & fl1577 \\\\\n\\midrule\ntime & 1.602132 & 1.077386 & 524.279018 & 4.400274 & 3.510462 \\\\\nobjective & 51112.000000 & 18691646.000000 & 1592268.000000 & 80883.000000 & 22305.000000 \\\\\noptimality_gaps & 0.006122 & 0.001686 & 0.012195 & 0.005382 & 0.002517 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [68]:
dataframe.loc['optimality_gaps']

d1291      0.006122
dsj1000    0.001686
d15112     0.012195
d2103      0.005382
fl1577     0.002517
Name: optimality_gaps, dtype: float64

## LKH-runs

In [114]:
task_name = 'LKH_runs_solutions/{}_param_sweep_LKH_time_limit_100000_max_trails_100000_max_runs_100.csv'
for task in tasks:
    name = task_name.format(task)
    if not os.path.exists(name):
        print(name, "not exist")

LKH_runs_solutions/d1291_param_sweep_LKH_time_limit_100000_max_trails_100000_max_runs_100.csv not exist
LKH_runs_solutions/d15112_param_sweep_LKH_time_limit_100000_max_trails_100000_max_runs_100.csv not exist


## ViTSP

Reasoning: GPT o4
- input: $1.1
- output: $4.4

Fast thinking: GPT 4.1
- input: $2
- output: $8


In [69]:
from helper.parse_instances import FileParser
import numpy as np

# calculate route
def cal_route_dist(route, task):
    file_parser = FileParser()
    instance_info = file_parser.parse_instance_from_file(f"instances/tsplib/{task}.tsp")
    node_coords = {i+1: np.array([x[0], x[1]]) for i, x in enumerate(instance_info["COORDINATES"])}

    total_dist = 0

    if instance_info['EDGE_WEIGHT_TYPE'] == "CEIL_2D":
        dist_func = lambda x1, x2: int(np.ceil(np.linalg.norm(x1 - x2)))
    elif instance_info['EDGE_WEIGHT_TYPE'] == "EUC_2D":
        dist_func = lambda x1, x2: int(np.sqrt(np.sum((x1 - x2) ** 2)) + 0.5)

    # if instance_info['EDGE_WEIGHT_TYPE'] == "CEIL_2D":
    #     dist_func = lambda x1, x2: np.ceil(np.linalg.norm(x1 - x2))
    # elif instance_info['EDGE_WEIGHT_TYPE'] == 'EUC_2D':
    #     dist_func = lambda x1, x2: np.floor(np.linalg.norm(x1 - x2) + 0.5)

    for i in range(len(route) - 1):
        node1 = node_coords[route[i]]
        if i == len(route) - 2:
            node2 = node_coords[route[0]]
        else:
            node2 = node_coords[route]
        total_dist += dist_func(node1, node2)

    return total_dist


In [70]:
task = 'dsj1000'
exp_name = "{}_max_nodes_1000_time_budget_100_initial_LKH_llm_gpt-4.1-2025-04-14_o4-mini-2025-04-16_solver_concorde_subproblem_2_parallel_workers"
task_name = task+ "_traces"
file = 'experiments/LLM_TSP_exp/' + exp_name.format(task_name) + ".txt"
route = [int(x.strip()) for x in open(file).readlines()[4:]]
cal_route_dist(route, task)

FileNotFoundError: [Errno 2] No such file or directory: 'experiments/LLM_TSP_exp/dsj1000_traces_max_nodes_1000_time_budget_100_initial_LKH_llm_gpt-4.1-2025-04-14_o4-mini-2025-04-16_solver_concorde_subproblem_2_parallel_workers.txt'

In [73]:
vitsp_times = []
vitsp_objs = []
vitsp_opt_gaps = []
exp_name = "{}_max_nodes_1000_time_budget_1000_initial_LKH_llm_gpt-4.1-2025-04-14_o4-mini-2025-04-16_solver_concorde_subproblem_2_parallel_workers.csv"
for task in tasks:
    filename = 'experiments/LLM_TSP_exp/' + exp_name.format(task)
    if not os.path.exists(filename):
        print(filename, "does not exist")
        vitsp_times.append(None)
        vitsp_objs.append(None)
        vitsp_opt_gaps.append(None)
        continue
    lines = [x.split(",") for x in open(filename).readlines()]
    objectives = [float(x[1]) for x in lines[2:]]
    times = [float(x[0]) for x in lines[2:]]
    argmin = 0
    objmin = objectives[0]
    for i in range(len(objectives)):
        if objectives[i] < objmin:
            argmin = i
            objmin = objectives[i]
    vitsp_times.append(times[argmin])
    vitsp_objs.append(objmin)
    opt = int(open('./instances/opt/{}.opt.tour.txt'.format(task)).readlines()[-1].strip())
    vitsp_opt_gaps.append((objmin - opt) / opt)

In [74]:
dataframe2 = pd.DataFrame([vitsp_times, vitsp_objs, vitsp_opt_gaps], columns=tasks, index=['time', 'objective', 'optimality_gaps'])
dataframe2

,d1291,dsj1000,d15112,d2103,fl1577
time,221.350000,115.03,1.975570e+03,597.740000,72.73
objective,50825.000000,18660188.00,1.590316e+06,80822.000000,22249.00
optimality_gaps,0.000472,0.00,1.095428e-02,0.004624,0.00


In [95]:
dataframe2.to_latex()

'\\begin{tabular}{lrrrrr}\n\\toprule\n & d1291 & dsj1000 & d15112 & d2103 & fl1577 \\\\\n\\midrule\ntime & 221.350000 & 115.030000 & 1975.570000 & 597.740000 & 72.730000 \\\\\nobjective & 50825.000000 & 18660188.000000 & 1590316.000000 & 80822.000000 & 22249.000000 \\\\\noptimality_gaps & 0.000472 & 0.000000 & 0.010954 & 0.004624 & 0.000000 \\\\\n\\bottomrule\n\\end{tabular}\n'

Reasoning: GPT 5
- input: $1.25
- output: $10

Fast thinking: GPT 5 mini
- input: $0.25
- output: $2


In [124]:
vitsp_times = []
vitsp_objs = []
vitsp_opt_gaps = []
exp_name = "{}_max_nodes_1000_time_budget_1000_initial_LKH_llm_gpt-5-mini-2025-08-07_gpt-5-2025-08-07_solver_concorde_subproblem_2_parallel_workers.csv"
for task in tasks:
    filename = 'experiments/LLM_TSP_exp-gpt5/' + exp_name.format(task)
    if not os.path.exists(filename):
        print(filename, "does not exist")
        vitsp_times.append(None)
        vitsp_objs.append(None)
        vitsp_opt_gaps.append(None)
        continue
    lines = [x.split(",") for x in open(filename).readlines()]
    objectives = [float(x[1]) for x in lines[1:]]
    times = [float(x[0]) for x in lines[1:]]
    argmin = 0
    objmin = objectives[0]
    for i in range(len(objectives)):
        if objectives[i] < objmin:
            argmin = i
            objmin = objectives[i]
    print(task, times[argmin], objmin)
    # if task == 'd15112':
        # objmin = 1590493
    vitsp_times.append(times[argmin])
    vitsp_objs.append(objmin)
    opt = int(open('./instances/opt/{}.opt.tour.txt'.format(task)).readlines()[-1].strip())
    vitsp_opt_gaps.append((objmin - opt) / opt)

d1291 779.34 51019.0
dsj1000 198.5 18668034.0
d15112 1999.93 1589647.0
d2103 359.17 80814.0
fl1577 172.9 22249.0


In [125]:
dataframe4 = pd.DataFrame([vitsp_times, vitsp_objs, vitsp_opt_gaps], columns=tasks, index=['time', 'objective', 'optimality_gaps'])
dataframe4

,d1291,dsj1000,d15112,d2103,fl1577
time,779.340000,1.985000e+02,1.999930e+03,359.170000,172.9
objective,51019.000000,1.866803e+07,1.589647e+06,80814.000000,22249.0
optimality_gaps,0.004291,4.204674e-04,1.052900e-02,0.004525,0.0


## Concorde

In [78]:
task2time = dataframe2.loc['time'].to_dict()

In [79]:
task2time

{'d1291': 221.35,
 'dsj1000': 115.03,
 'd15112': 1975.57,
 'd2103': 597.74,
 'fl1577': 72.73}

In [80]:
from exact_concorde.exact_concorde import *
import os
os.environ["QSOPT_DIR"] = os.path.abspath("../pyconcorde/data/")

def run_concorde(task, t):
    file_parser = FileParser()
    solution_plotter = SolutionPlot()

    fname = './instances/tsplib/{}.tsp'.format(task)
    print("running:", fname)
    
    instance_info = file_parser.parse_instance_from_file(fname)
    coordinates = instance_info['COORDINATES']
    nodes = {i: (x, y) for i, (x, y) in enumerate(coordinates)}
    tsp_instance = TravelingSalesmenProblem(node_coords_dict=nodes)

    t1 = time.time()
    concorde_model = Concorde(nodes=list(nodes.keys()), coordinates=coordinates)
    concorde_model.optimize(timelimit=t, verbose=False)

    current_route = concorde_model.get_tsp_route()
    current_obj = concorde_model.get_objective_value()
    t2 = time.time()

    return current_obj, t2 - t1

concorde_objs = []
concorde_ts = []
concorde_opt_gaps = []

for task, t in task2time.items():
    obj, concorde_time = run_concorde(task, t)
    opt = int(open('./instances/opt/{}.opt.tour.txt'.format(task)).readlines()[-1].strip())
    concorde_objs.append(obj)
    concorde_ts.append(concorde_time)
    concorde_opt_gaps.append((obj - opt) / opt)



[WARNING] CuPy not found. Falling back to NumPy.
running: ./instances/tsplib/d1291.tsp


100%|█████████▉| 1297/1298 [00:00<00:00, 1104795.35it/s]
need to link an lp solver to use this function
CClp_getweight failed
find_candidate_cliques failed
need to link an lp solver to use this function
CClp_getweight failed
find_candidate_cliques failed
Hit time limit in bfs branching


Problem Name: Pseudo_TSP_Instance
Problem Type: TSP
Number of Nodes: 1291
Rounded Euclidean Norm (CC_EUCLIDEAN)
CCtsp_solve_dat ...
Finding a good tour for compression ...
linkern ...
Starting Cycle: 63271
   0 Steps   Best: 55017   0.00 seconds
   5 Steps   Best: 54943   0.00 seconds
   7 Steps   Best: 54922   0.00 seconds
  12 Steps   Best: 53659   0.00 seconds
  15 Steps   Best: 53636   0.00 seconds
  20 Steps   Best: 53403   0.01 seconds
  26 Steps   Best: 53380   0.01 seconds
  33 Steps   Best: 53378   0.01 seconds
  44 Steps   Best: 52069   0.01 seconds
  58 Steps   Best: 52056   0.01 seconds
  63 Steps   Best: 52051   0.01 seconds
  67 Steps   Best: 52002   0.01 seconds
  77 Steps   Best: 51986   0.01 seconds
  79 Steps   Best: 51971   0.01 seconds
  94 Steps   Best: 51934   0.01 seconds
 103 Steps   Best: 51923   0.01 seconds
 106 Steps   Best: 51891   0.01 seconds
 116 Steps   Best: 51848   0.01 seconds
 130 Steps   Best: 51639   0.01 seconds
 155 Steps   Best: 51636   0.02 se

100%|█████████▉| 1006/1007 [00:00<00:00, 1131225.15it/s]

Problem Name: Pseudo_TSP_Instance
Problem Type: TSP
Number of Nodes: 1000
Rounded Euclidean Norm (CC_EUCLIDEAN)


...

Problem Name: Pseudo_TSP_Instance
Problem Type: TSP
Number of Nodes: 15112
Rounded Euclidean Norm (CC_EUCLIDEAN)
CCtsp_solve_dat ...
Finding a good tour for compression ...
linkern ...
Starting Cycle: 1798570
   0 Steps   Best: 1597302   0.05 seconds
   2 Steps   Best: 1596968   0.05 seconds
   4 Steps   Best: 1596904   0.05 seconds
   5 Steps   Best: 1596330   0.05 seconds
   6 Steps   Best: 1596251   0.05 seconds
   7 Steps   Best: 1596062   0.05 seconds
   8 Steps   Best: 1595908   0.05 seconds
  10 Steps   Best: 1595688   0.05 seconds
  12 Steps   Best: 1595659   0.05 seconds
  13 Steps   Best: 1595633   0.05 seconds
  17 Steps   Best: 1595624   0.05 seconds
  18 Steps   Best: 1595599   0.05 seconds
  19 Steps   Best: 1595580   0.05 seconds
  20 Steps   Best: 1595553   0.05 seconds
  21 Steps   Best: 1595526   0.06 seconds
  22 Steps   Best: 1595415   0.06 seconds
  24 Steps   Best: 1595393   0.06 seconds
  25 Steps   Best: 1595174   0.06 seconds
  26 Steps   Best: 1595000   0.06 s

100%|█████████▉| 2109/2110 [00:00<00:00, 1039946.76it/s]
No acceptable pivot found


Problem Name: Pseudo_TSP_Instance
Problem Type: TSP
Number of Nodes: 2103
Rounded Euclidean Norm (CC_EUCLIDEAN)
CCtsp_solve_dat ...
Finding a good tour for compression ...
linkern ...
Starting Cycle: 92091
   0 Steps   Best: 82071   0.00 seconds
   3 Steps   Best: 82060   0.00 seconds
   8 Steps   Best: 82054   0.00 seconds
  11 Steps   Best: 82048   0.00 seconds
  15 Steps   Best: 82022   0.00 seconds
  17 Steps   Best: 82007   0.00 seconds
  21 Steps   Best: 81959   0.00 seconds
  30 Steps   Best: 81947   0.01 seconds
  33 Steps   Best: 81936   0.01 seconds
  35 Steps   Best: 81918   0.01 seconds
  36 Steps   Best: 81870   0.01 seconds
  39 Steps   Best: 81866   0.01 seconds
  42 Steps   Best: 81862   0.01 seconds
  43 Steps   Best: 81821   0.01 seconds
  50 Steps   Best: 81801   0.01 seconds
  60 Steps   Best: 81792   0.01 seconds
  63 Steps   Best: 81790   0.01 seconds
  66 Steps   Best: 81786   0.01 seconds
  91 Steps   Best: 81764   0.01 seconds
 104 Steps   Best: 81763   0.02 se

100%|█████████▉| 1583/1584 [00:00<00:00, 1117963.16it/s]


Problem Name: Pseudo_TSP_Instance
Problem Type: TSP
Number of Nodes: 1577
Rounded Euclidean Norm (CC_EUCLIDEAN)
CCtsp_solve_dat ...
Finding a good tour for compression ...
linkern ...
Starting Cycle: 25214
   0 Steps   Best: 23792   0.00 seconds
   4 Steps   Best: 23784   0.00 seconds
  13 Steps   Best: 23782   0.01 seconds
  22 Steps   Best: 23777   0.01 seconds
  31 Steps   Best: 23754   0.01 seconds
  32 Steps   Best: 23753   0.01 seconds
  51 Steps   Best: 23749   0.02 seconds
  52 Steps   Best: 23738   0.02 seconds
  53 Steps   Best: 23733   0.02 seconds
  54 Steps   Best: 23110   0.02 seconds
  55 Steps   Best: 23052   0.02 seconds
  60 Steps   Best: 23051   0.02 seconds
  98 Steps   Best: 23047   0.03 seconds
 114 Steps   Best: 23045   0.03 seconds
 133 Steps   Best: 23042   0.04 seconds
 137 Steps   Best: 23040   0.04 seconds
 138 Steps   Best: 23036   0.04 seconds
 139 Steps   Best: 23033   0.04 seconds
 172 Steps   Best: 23025   0.05 seconds
 223 Steps   Best: 23024   0.06 se

In [81]:
dataframe3 = pd.DataFrame([concorde_ts, concorde_objs, concorde_opt_gaps], columns=tasks, index=['time', 'objective', 'optimality_gaps'])
dataframe3

,d1291,dsj1000,d15112,d2103,fl1577
time,251.765765,3.448656e+01,2.113479e+03,3.704892e+03,1.229237e+02
objective,50872.000000,1.865969e+07,1.577228e+06,1.000000e+30,1.000000e+30
optimality_gaps,0.001398,-2.679501e-05,2.634316e-03,1.243008e+25,4.494584e+25


In [83]:
task2time

{'d1291': 221.35,
 'dsj1000': 115.03,
 'd15112': 1975.57,
 'd2103': 597.74,
 'fl1577': 72.73}

In [84]:
task = 'fl1577'
obj, t = run_concorde(task, task2time[task])

running: ./instances/tsplib/fl1577.tsp


100%|█████████▉| 1583/1584 [00:00<00:00, 737977.46it/s]


Problem Name: Pseudo_TSP_Instance
Problem Type: TSP
Number of Nodes: 1577
Rounded Euclidean Norm (CC_EUCLIDEAN)
CCtsp_solve_dat ...
Finding a good tour for compression ...
linkern ...
Starting Cycle: 25214
   0 Steps   Best: 23792   0.00 seconds
   4 Steps   Best: 23784   0.00 seconds
  13 Steps   Best: 23782   0.01 seconds
  22 Steps   Best: 23777   0.01 seconds
  31 Steps   Best: 23754   0.01 seconds
  32 Steps   Best: 23753   0.01 seconds
  51 Steps   Best: 23749   0.01 seconds
  52 Steps   Best: 23738   0.02 seconds
  53 Steps   Best: 23733   0.02 seconds
  54 Steps   Best: 23110   0.02 seconds
  55 Steps   Best: 23052   0.02 seconds
  60 Steps   Best: 23051   0.02 seconds
  98 Steps   Best: 23047   0.03 seconds
 114 Steps   Best: 23045   0.03 seconds
 133 Steps   Best: 23042   0.04 seconds
 137 Steps   Best: 23040   0.04 seconds
 138 Steps   Best: 23036   0.04 seconds
 139 Steps   Best: 23033   0.04 seconds
 172 Steps   Best: 23025   0.04 seconds
 223 Steps   Best: 23024   0.06 se

In [100]:
dataframe3

,d1291,dsj1000,d15112,d2103,fl1577
time,251.765765,3.448656e+01,2.113479e+03,3.704892e+03,1.229237e+02
objective,50872.000000,1.865969e+07,1.577228e+06,1.000000e+30,1.000000e+30
optimality_gaps,0.001398,-2.679501e-05,2.634316e-03,1.243008e+25,4.494584e+25


In [99]:
dataframe3.loc["time", 'fl1577']

np.float64(122.9236752986908)

In [86]:
t

115.03366017341614

In [ ]:
task = 'fl1577'
obj, t = run_concorde(task, task2time[task])

In [102]:
dataframe3.to_latex()

'\\begin{tabular}{lrrrrr}\n\\toprule\n & d1291 & dsj1000 & d15112 & d2103 & fl1577 \\\\\n\\midrule\ntime & 251.765765 & 34.486558 & 2113.478674 & 3704.892238 & 122.923675 \\\\\nobjective & 50872.000000 & 18659688.000000 & 1577228.000000 & 1000000000000000019884624838656.000000 & 1000000000000000019884624838656.000000 \\\\\noptimality_gaps & 0.001398 & -0.000027 & 0.002634 & 12430080795525171413778432.000000 & 44945840262483706264944640.000000 \\\\\n\\bottomrule\n\\end{tabular}\n'